### Connect to Database and Load Data


In [1]:
import sqlite3
import pandas as pd

In [2]:
%load_ext sql
con = sqlite3.connect('./database/spacex.db')
cur = con.cursor()

In [3]:
%sql sqlite:///database/spacex.db
df = pd.read_csv("./data/spacex_IBM_data.csv")
df.to_sql('SPACEXTBL', con, if_exists='replace', index=False,method='multi')

101

In [4]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///database/spacex.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### SQL Queries


In [5]:
# Display the names of the unique launch sites in the space mission
%sql SELECT DISTINCT launch_site AS "Launch Sites" FROM SPACEXTBL

 * sqlite:///database/spacex.db
Done.


Launch Sites
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [6]:
# Display 5 records where launch sites begin with the string CCA
%sql SELECT * FROM SPACEXTBL WHERE launch_site LIKE "CCA%" LIMIT 5

 * sqlite:///database/spacex.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-08-12,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-08-10,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-01-03,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [7]:
# Display the total payload mass carried by boosters launched by NASA (CRS)
%sql SELECT SUM(payload_mass__kg_) AS "Total Payload Mass" FROM SPACEXTBL WHERE "Customer" = "NASA (CRS)"

 * sqlite:///database/spacex.db
Done.


Total Payload Mass
45596


In [8]:
# Display average payload mass carried by booster version F9 v1.1
%sql SELECT AVG(payload_mass__kg_) AS "Average Payload Mass" FROM SPACEXTBL WHERE booster_version = "F9 v1.1"

 * sqlite:///database/spacex.db
Done.


Average Payload Mass
2928.4


In [9]:
# List the date when the first succesful landing outcome in ground pad was acheived
%sql SELECT MIN(DATE) AS "First Successful Landing" FROM SPACEXTBL WHERE Landing_Outcome = "Success (ground pad)"

 * sqlite:///database/spacex.db
Done.


First Successful Landing
2015-12-22


In [10]:
# List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000
%sql SELECT booster_version FROM SPACEXTBL WHERE Landing_Outcome = "Success (drone ship)" AND payload_mass__kg_ BETWEEN 4000 AND 6000

 * sqlite:///database/spacex.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [11]:
# List the total number of successful and failure mission outcomes
%sql SELECT COUNT(*) AS "Successful", (SELECT COUNT(*) FROM SPACEXTBL WHERE Mission_Outcome LIKE 'Failure%') AS "Failure" FROM SPACEXTBL WHERE Mission_Outcome LIKE "Success%"

 * sqlite:///database/spacex.db
Done.


Successful,Failure
100,1


In [12]:
# List the names of the booster_versions which have carried the maximum payload mass
%sql SELECT booster_version, payload_mass__kg_ FROM SPACEXTBL WHERE payload_mass__kg_ = (SELECT MAX(payload_mass__kg_) FROM SPACEXTBL)

 * sqlite:///database/spacex.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


In [13]:
# List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015
%sql SELECT Landing_Outcome, booster_version, launch_site FROM SPACEXTBL WHERE DATE LIKE "2015%" AND Landing_Outcome LIKE "Failure (drone ship)"

 * sqlite:///database/spacex.db
Done.


Landing_Outcome,Booster_Version,Launch_Site
Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [14]:
# Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order
%sql SELECT Landing_Outcome, COUNT(*) AS "Count" FROM SPACEXTBL WHERE DATE BETWEEN "2010-06-04" AND "2017-03-20" GROUP BY Landing_Outcome ORDER BY "Count" DESC

 * sqlite:///database/spacex.db
Done.


Landing_Outcome,Count
No attempt,10
Success (ground pad),5
Success (drone ship),5
Failure (drone ship),5
Controlled (ocean),3
Uncontrolled (ocean),2
Precluded (drone ship),1
Failure (parachute),1
